In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp drive/MyDrive/ssne/train.pkl train.pkl
!cp drive/MyDrive/ssne/test_no_target.pkl test_no_target.pkl

In [3]:
import pandas as pd
import sklearn as sklearn
import torch
import random
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [4]:
device = torch.device("cuda")
device

device(type='cuda')

In [5]:
seed = 742842589
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [6]:
train_raw = pd.read_pickle("./train.pkl")
random.shuffle(train_raw)

In [7]:
def count_classes(train_set):
  classes_count = {}
  for x in train_set:
    classes_count[x[1]] = classes_count.get(x[1], 0) + 1
  return classes_count

print(f"Classes: {count_classes(train_raw)}")
min_class_count = min(count_classes(train_raw).values())
print(f"Min class: {min_class_count}")

Classes: {0: 1630, 2: 154, 3: 441, 4: 236, 1: 478}
Min class: 154


In [8]:
# Balancing
CLASS_COUNT_LIMIT = 5*min_class_count

train_balanced = []
classes_counter = {}
for x in train_raw:
  classes_counter[x[1]] = classes_counter.get(x[1], 0) + 1
  if classes_counter[x[1]] <= CLASS_COUNT_LIMIT:
    train_balanced.append(x)

train_raw = train_balanced
print(f"Classes: {count_classes(train_raw)}")

Classes: {0: 770, 2: 154, 3: 441, 4: 236, 1: 478}


In [9]:
sizes = []
for x in train_raw:
  sizes.append(x[0].shape[0])
print(f"Sequence size [min/avr/max]: {min(sizes)}/{sum(sizes)/len(sizes)}/{max(sizes)}")

Sequence size [min/avr/max]: 4/473.93265993265993/6308


In [10]:
# Croping
MAX_SEQUENCE_LEN = 40000

music_data = []
label_data = []
for x in train_raw:
  music_tensor = torch.tensor(x[0], dtype=torch.float)
  music_tensor = music_tensor[:MAX_SEQUENCE_LEN]
  music_data.append(music_tensor)
  label_data.append(x[1])
music_data[0]

tensor([ -1.,  -1.,   0.,   0.,   0.,  12.,  12.,  88.,  92.,  92.,  28., 159.,
         12.,   0.,   0., 144., 144., 144.,  50.,  50.,  50., 116., 116., 116.,
        122., 122., 122., 127.,  88.,  88., 125.,  78., 119., 159., 125.,  47.,
         78.,  64., 159.,  92.,  88.,  47.,  12.,   0.,  12.,  12.,  15.,  45.,
        124., 112.,  12.,  13., 141.,  12.,  88.,  13.,  92.,  92.,   0.,   0.])

In [11]:
music_data_padded = pad_sequence(music_data, batch_first=True, padding_value=0)
assert len(music_data_padded) == len(label_data)
print(f"Size after paddding: {music_data_padded[0].shape[0]}")

Size after paddding: 6308


In [12]:
TRAIN_TEST_RATIO = 0.9
BATCH_SIZE = 32

train_indices = int(TRAIN_TEST_RATIO*len(music_data_padded))

train_set = torch.utils.data.TensorDataset(music_data_padded[:train_indices], torch.tensor(label_data[:train_indices]))
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE)

test_data, test_targets = music_data_padded[train_indices:], label_data[train_indices:]

In [13]:
class LSTMRegressor(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, out_size):
    super().__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)
    self.fc = nn.Linear(hidden_size*len(music_data_padded[0]), out_size)

  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
    state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
    return hidden, state

  def forward(self, x, hidden):
    x = torch.transpose(x, 0, 1)
    all_outputs, hidden = self.lstm(x, hidden)
    all_outputs = torch.transpose(all_outputs,0,1)
    out = torch.flatten(all_outputs, 1)
    x = self.fc(out)
  
    return x, hidden

In [14]:
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(label_data),y=label_data)
class_weights = torch.tensor(class_weights).float()

In [15]:
def count_accuracy():
  with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device)
    preds, _ = model(test_data.to(device).unsqueeze(2),(hidden, state))
    p = torch.argmax(preds,1).cpu()
    counter = 0
    for i in range(len(test_targets)):
      if p[i] == test_targets[i]:
        counter += 1
    return counter/len(test_targets)

In [ ]:
model = LSTMRegressor(1, 50, 3, 5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss(weight = class_weights.to(device))

In [17]:
model.train()
max_acc = 0
for epoch in range(6):
  for x, targets in train_loader:
    x = x.to(device).unsqueeze(2)
    targets = targets.to(device)
    hidden, state = model.init_hidden(x.size(0))
    hidden, state = hidden.to(device), state.to(device)
    preds, _ = model(x, (hidden,state))
    preds = preds.squeeze(1)
    optimizer.zero_grad()
    loss = loss_fun(preds, targets)
    loss.backward()
    optimizer.step()
  acc = count_accuracy()
  max_acc = max(max_acc, acc)
  if max_acc - acc > 0.05:
    break
  print(f"Epoch: {epoch}, loss: {loss.item():.3}, acc: {acc:.3}, max_acc: {max_acc:.3}")

Epoch: 0, loss: 1.02, acc: 0.567, max_acc: 0.567
Epoch: 1, loss: 0.828, acc: 0.639, max_acc: 0.639
Epoch: 2, loss: 0.898, acc: 0.654, max_acc: 0.654
Epoch: 3, loss: 0.69, acc: 0.663, max_acc: 0.663
Epoch: 4, loss: 0.667, acc: 0.688, max_acc: 0.688
Epoch: 5, loss: 0.621, acc: 0.688, max_acc: 0.688
Epoch: 6, loss: 0.536, acc: 0.606, max_acc: 0.688
Epoch: 7, loss: 0.429, acc: 0.596, max_acc: 0.688


KeyboardInterrupt: ignored

In [ ]:
def save_results(tensor):
  predictions = tensor.cpu().detach().numpy()
  pd.DataFrame(predictions).to_csv("result.csv",header=False, index=False)

In [ ]:
test_raw = pd.read_pickle("./test_no_target.pkl")